In [30]:
import pandas as pd
import glob
import os
import numpy as np
from pathlib import Path
from datetime import datetime

## Tính tỷ lệ FF

In [27]:
import glob
import os
import pandas as pd
from pathlib import Path
from datetime import datetime

def process_free_float_data(tennganh, external_market_cap_file=None):
    folder_path = Path(f"D:\\nckh\\data\\{tennganh}")
    excel_files = glob.glob(str(folder_path / "*.xlsx"))

    if not excel_files:
        print(f"No .xlsx files found in {folder_path}")
        return None

    dataframes = []
    for file in excel_files:
        print(f"Reading {file}...")
        try:
            df = pd.read_excel(file, header=7)
            df['SourceFile'] = os.path.basename(file)
            dataframes.append(df)
        except (pd.errors.ParserError, FileNotFoundError) as e:
            print(f"Error reading {file}: {e}")
            continue

    if not dataframes:
        print(f"No valid DataFrames created from files in {folder_path}")
        return None

    df = pd.concat(dataframes, ignore_index=True)

    # Drop rows with all critical metadata missing
    initial_rows = len(df)
    df = df.dropna(subset=['STT','Mã', 'Tên công ty', 'Ngày'], how='all')
    print(f"Dropped {initial_rows - len(df)} rows with missing Stock_Code, Company_Name, and Date.")

    # Standardize column names
    df = df.rename(columns={
        'Số CP Free Float\nĐơn vị: Cổ phiếu': 'Free_Float',
        'Số CP lưu hành hiện thời\nĐơn vị: Cổ phiếu': 'Luu_Hanh',
        'Số CP niêm yết\nĐơn vị: Cổ phiếu': 'Niem_Yet',
        'Ngày': 'Date',
        'Mã': 'Stock_Code',
        'Tên công ty': 'Company_Name',
        'Vốn hóa thị trường\nĐơn vị: VND': 'Market_Cap',
        'STT': 'Index',
        'SourceFile': 'Source_File'
    })

    # Convert data types
    numeric_cols = ['STT','Free_Float', 'Luu_Hanh', 'Niem_Yet', 'Market_Cap']
    for col in numeric_cols:
        if col in df.columns:
            df[col] = pd.to_numeric(df[col], errors='coerce')
    df['Date'] = pd.to_datetime(df['Date'], errors='coerce')

    # Merge external Market_Cap if provided
    if external_market_cap_file:
        print(f"Loading external Market_Cap from {external_market_cap_file}")
        external_data = pd.read_csv(external_market_cap_file)
        external_data['Date'] = pd.to_datetime(external_data['Date'], errors='coerce')
        external_data = external_data[['Stock_Code', 'Date', 'Market_Cap']]
        df = df.drop(columns=['Market_Cap'], errors='ignore')
        df = df.merge(external_data, on=['Stock_Code', 'Date'], how='left')
        print("\n=== External Market_Cap Merged ===")
        print(df[['Stock_Code', 'Date', 'Market_Cap']].head())

    # Print initial structure
    print("\n=== First 5 rows ===")
    print(df.head())
    print("\n=== Column names and info ===")
    print(df.info())

    # Log missing data
    print("\n=== Missing Data Summary ===")
    for col in df.columns:
        missing_count = df[col].isna().sum()
        if missing_count > 0:
            print(f"Column {col}: {missing_count} missing values")

    # Investigate missing Free_Float
    if df['Free_Float'].isna().any():
        print("\n=== Stocks with Missing Free_Float ===")
        print(df[df['Free_Float'].isna()]['Stock_Code'].value_counts().head())
        print("\n=== Dates with Missing Free_Float ===")
        print(df[df['Free_Float'].isna()]['Date'].value_counts().sort_index().head())

    # Sort by Stock_Code and Date for consistent filling
    df = df.sort_values(['Stock_Code', 'Date'])

    # Apply ffill within Stock_Code
    for col in ['Free_Float', 'Luu_Hanh', 'Market_Cap']:
        if col in df.columns and df[col].isna().any():
            print(f"Applying ffill to {col} within Stock_Code.")
            df[col] = df.groupby('Stock_Code')[col].ffill()

    # Drop remaining missing values
    missing_free_float = df['Free_Float'].isna().sum()
    missing_market_cap = df['Market_Cap'].isna().sum()
    missing_luu_hanh = df['Luu_Hanh'].isna().sum()
    if missing_free_float > 0 or missing_market_cap > 0 or missing_luu_hanh > 0:
        print(f"Dropping {missing_free_float} rows with missing Free_Float, {missing_market_cap} rows with missing Market_Cap, and {missing_luu_hanh} rows with missing Luu_Hanh.")
        df = df.dropna(subset=['Free_Float', 'Market_Cap', 'Luu_Hanh'])

    # Handle invalid dates
    invalid_dates = df['Date'].isna().sum()
    if invalid_dates > 0:
        print(f"Dropping {invalid_dates} rows with invalid dates.")
        df = df.dropna(subset=['Date'])

    # Filter invalid data
    df_long = df[df['Luu_Hanh'] > 0]
    dropped_luu_hanh = len(df) - len(df_long)
    if dropped_luu_hanh > 0:
        print(f"Dropped {dropped_luu_hanh} rows with Luu_Hanh <= 0.")
    df_long = df_long[df_long['Free_Float'] <= df_long['Luu_Hanh']]
    df_long = df_long[df_long['Free_Float'] >= 0]

    # Log data retention
    print(f"\nRemaining rows after filtering: {len(df_long)} ({len(df_long)/initial_rows*100:.2f}% of original)")
    print(f"Unique stocks: {df_long['Stock_Code'].nunique()}")
    print(f"Unique dates: {df_long['Date'].nunique()}")

    print("\n=== After Pre-Processing ===")
    print(df_long.head())
    print("\n=== Debug Data ===")
    print("Total Free_Float = 0:", (df_long['Free_Float'] == 0).sum())
    print("Total Luu_Hanh = 0:", (df_long['Luu_Hanh'] == 0).sum())
    print("Free_Float sum by Date:", df_long.groupby('Date')['Free_Float'].sum().head())
    print("Luu_Hanh sum by Date:", df_long.groupby('Date')['Luu_Hanh'].sum().head())
    print("\n=== Missing Data After Preprocessing ===")
    print(df_long.isna().sum())

    # Calculate industry Free Float ratio
    nghanh_freefloat = df_long.groupby('Date').apply(
        lambda x: x['Free_Float'].sum() / x['Luu_Hanh'].sum() if x['Luu_Hanh'].sum() != 0 else 0
    )
    zero_luu_hanh_dates = df_long.groupby('Date').filter(lambda x: x['Luu_Hanh'].sum() == 0)['Date'].unique()
    for date in zero_luu_hanh_dates:
        print(f"Warning: Zero Luu_Hanh sum for date {date}")
    nghanh_freefloat = pd.DataFrame(nghanh_freefloat, columns=['Tỷ lệ FF']).reset_index()

    # # Calculate adjusted market cap
    # df_long = df_long.merge(nghanh_freefloat, on='Date', how='left')
    # df_long['Adjusted_Market_Cap'] = df_long['Market_Cap'] * df_long['Tỷ lệ FF']

    # # Print results
    # print(f"\n=== TỶ LỆ FREE FLOAT NGÀNH ({tennganh}) ===")
    # print(nghanh_freefloat.head(10))
    # print("\n=== Adjusted Market Cap (First 5 rows) ===")
    # print(df_long[['Date', 'Stock_Code', 'Market_Cap', 'Tỷ lệ FF', 'Adjusted_Market_Cap']].head())

    # Save results with timestamp
    output = f'ty_le_freefloat_{tennganh}'
    output_path = Path("data") / "processed" / f"{output}.xlsx"
    os.makedirs(output_path.parent, exist_ok=True)
    nghanh_freefloat.to_excel(output_path, index=False)

    # output_adj = f'adjusted_market_cap_{tennganh}_{timestamp}'
    # output_adj_path = Path("data") / "processed" / f"{output_adj}.xlsx"
    # df_long[['Date', 'Stock_Code', 'Company_Name', 'Free_Float', 'Luu_Hanh', 'Market_Cap', 'Tỷ lệ FF', 'Adjusted_Market_Cap']].to_excel(output_adj_path, index=False)

    return df_long, nghanh_freefloat

if __name__ == "__main__":
    list_nganh = ['hangtieudung', 'duocphamyte', 'taichinh', 'nganhang', 'congnghiep', 
                  'dichvutieudung', 'tienichcd', 'nguyenvatlieu', 'vienthong', 'cntt']
    # Replace with the path to your external Market_Cap file
    # external_file = "D:\\nckh\\data\\external_market_cap.csv"  # Update this path
    for nganh in list_nganh:
        result_df, result_ff = process_free_float_data(nganh)
        if result_df is not None:
            print(f"Processing completed. {nganh} results saved to files.")

Reading D:\nckh\data\hangtieudung\hangtieudung_1.xlsx...
Reading D:\nckh\data\hangtieudung\hangtieudung_2.xlsx...
Reading D:\nckh\data\hangtieudung\hangtieudung_3.xlsx...
Reading D:\nckh\data\hangtieudung\hangtieudung_4.xlsx...
Reading D:\nckh\data\hangtieudung\hangtieudung_5.xlsx...
Reading D:\nckh\data\hangtieudung\hangtieudung_6.xlsx...
Reading D:\nckh\data\hangtieudung\hangtieudung_7.xlsx...
Dropped 21 rows with missing Stock_Code, Company_Name, and Date.

=== First 5 rows ===
  Index Stock_Code               Company_Name       Date   Free_Float  \
0     1        CTF                  City Auto 2025-10-01   43044236.0   
1     2        CTP            Hoà Bình Takara 2025-10-01    7864994.0   
2     3        DAS   Thiết bị Dầu khí Đà Nẵng 2025-10-01    2310000.0   
3     4        DAT  ĐT Du lịch và PT Thủy sản 2025-10-01    8999684.0   
4     5        DBC            Tập đoàn DABACO 2025-10-01  250163333.0   

      Luu_Hanh     Niem_Yet    Market_Cap          Source_File  
0   956538

C:\Users\caida\AppData\Local\Temp\ipykernel_7376\3003727649.py:135: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  nghanh_freefloat = df_long.groupby('Date').apply(


Processing completed. hangtieudung results saved to files.
Reading D:\nckh\data\duocphamyte\duocphamyte1.xlsx...
Reading D:\nckh\data\duocphamyte\duocphamyte2.xlsx...
Dropped 6 rows with missing Stock_Code, Company_Name, and Date.

=== First 5 rows ===
  Index Stock_Code           Company_Name       Date   Free_Float  \
0     1        AGP  Dược phẩm AGIMEXPHARM 2025-10-01   11976871.0   
1     2        AMP              Armephaco 2025-10-01    1950000.0   
2     3        AMV   Dược-TB Y tế Việt Mỹ 2025-10-01  131105650.0   
3     4        APC        Chiếu xạ An Phú 2025-10-01   19905025.0   
4     5        BCH          Dược Bảo Châu 2025-10-01          0.0   

      Luu_Hanh    Market_Cap     Niem_Yet        Source_File  
0   26615268.0  1.102910e+12   26615268.0  duocphamyte1.xlsx  
1   13000000.0  1.729000e+11   13000000.0  duocphamyte1.xlsx  
2  131105650.0  2.622113e+11  131105650.0  duocphamyte1.xlsx  
3   19905025.0  1.498052e+11   20121395.0  duocphamyte1.xlsx  
4   21500000.0   

C:\Users\caida\AppData\Local\Temp\ipykernel_7376\3003727649.py:135: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  nghanh_freefloat = df_long.groupby('Date').apply(


Processing completed. duocphamyte results saved to files.
Reading D:\nckh\data\taichinh\taichinh1.xlsx...
Reading D:\nckh\data\taichinh\taichinh2.xlsx...
Reading D:\nckh\data\taichinh\taichinh3.xlsx...
Reading D:\nckh\data\taichinh\taichinh4.xlsx...
Reading D:\nckh\data\taichinh\taichinh5.xlsx...
Reading D:\nckh\data\taichinh\taichinh6.xlsx...
Dropped 18 rows with missing Stock_Code, Company_Name, and Date.

=== First 5 rows ===
  Index Stock_Code             Company_Name       Date   Free_Float  \
0     1        AAS  Chứng khoán SmartInvest 2025-10-20  229999521.0   
1     2        AAV     Việt Tiên Sơn Địa ốc 2025-10-20   51740745.0   
2     3        ABI        BH NH Nông Nghiệp 2025-10-20   35471671.0   
3     4        ABW      Chứng khoán An Bình 2025-10-20   25287500.0   
4     5       ACBS          Chứng khoán ACB 2025-10-20          0.0   

       Luu_Hanh      Niem_Yet  Market_Cap     Source_File  
0  2.299995e+08  2.299995e+08         NaN  taichinh1.xlsx  
1  6.898766e+07  6.8

...

Processing completed. nguyenvatlieu results saved to files.
Reading D:\nckh\data\vienthong\vienthong.xlsx...
Dropped 3 rows with missing Stock_Code, Company_Name, and Date.

=== First 5 rows ===
  Index Stock_Code                            Company_Name       Date  \
0     1        ABC                        Truyền thông VMG 2025-10-01   
1     2        FOX                             FPT Telecom 2025-10-01   
2     3        MFS                        Mobifone Service 2025-10-01   
3     4        PAI  CNTT, VT và Tự động hóa Dầu khí - PAIC 2025-10-01   
4     5        PIA           Tin học Viễn thông Petrolimex 2025-10-01   

   Free_Float     Luu_Hanh     Niem_Yet    Market_Cap     Source_File  
0  12233400.0   20389000.0   20393000.0  2.192225e+11  vienthong.xlsx  
1  30802610.0  738763463.0  738763463.0  4.755568e+13  vienthong.xlsx  
2   3884638.0    7062979.0    7062979.0  2.792561e+11  vienthong.xlsx  
3     82650.0    4235290.0    4235290.0  4.870584e+10  vienthong.xlsx  
4   13

C:\Users\caida\AppData\Local\Temp\ipykernel_7376\3003727649.py:135: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  nghanh_freefloat = df_long.groupby('Date').apply(


Processing completed. vienthong results saved to files.
Reading D:\nckh\data\cntt\CNTT.xlsx...
Dropped 3 rows with missing Stock_Code, Company_Name, and Date.

=== First 5 rows ===
  Index Stock_Code             Company_Name       Date    Free_Float  \
0     1        CKV                 CokyVina 2025-10-01  1.604800e+06   
1     2        CMG   Tập đoàn Công nghệ CMC 2025-10-01  8.469804e+07   
2     3        CMT  CN mạng và Truyền thông 2025-10-01  2.911112e+06   
3     4        ELC                    ELCOM 2025-10-01  7.339320e+07   
4     5        FPT                 FPT Corp 2025-10-01  1.447981e+09   

       Luu_Hanh      Niem_Yet    Market_Cap Source_File  
0  4.012000e+06  4.050000e+06  4.734160e+10   CNTT.xlsx  
1  2.117451e+08  2.117839e+08  8.289821e+12   CNTT.xlsx  
2  7.277780e+06  8.000000e+06  1.001568e+11   CNTT.xlsx  
3  1.048474e+08  1.048474e+08  2.369552e+12   CNTT.xlsx  
4  1.703507e+09  1.703507e+09  1.591076e+14   CNTT.xlsx  

=== Column names and info ===
<class 

C:\Users\caida\AppData\Local\Temp\ipykernel_7376\3003727649.py:135: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  nghanh_freefloat = df_long.groupby('Date').apply(


Processing completed. cntt results saved to files.


In [28]:
# def process_free_float_data(tennganh):
#     folder_path = f"data\\raw\{tennganh}"
#     excel_files = glob.glob(os.path.join(folder_path, "*.xlsx"))

#     if not excel_files:
#         print(f"No .xlsx files found in {folder_path}")
#         return None

#     dataframes = []

#     for file in excel_files:
#         print(f"Reading {file}...")
#         try:
#             df = pd.read_excel(file, header=7)  # 8th row = header
#             df['SourceFile'] = os.path.basename(file)
#             dataframes.append(df)
#         except Exception as e:
#             print(f"Error reading {file}: {e}")
#             continue

#     if not dataframes:
#         print(f"No valid DataFrames created from files in {folder_path}")
#         return None

#     # Concatenate all DataFrames
#     df = pd.concat(dataframes, ignore_index=True)
#     df = pd.DataFrame(df)

#     # Print initial structure
#     print("=== First 5 rows ===")
#     print(df.head())
#     print("\n=== Column names and info ===")
#     print(df.info())

#     # 2. CHUẨN HÓA TÊN CỘT
#     df = df.rename(columns={
#         'Số CP Free Float\nĐơn vị: Cổ phiếu': 'Free_Float',
#         'Số CP lưu hành hiện thời\nĐơn vị: Cổ phiếu': 'Luu_Hanh',
#         'Số CP niêm yết\nĐơn vị: Cổ phiếu': 'Niem_Yet',
#         'Ngày': 'Date',
#         'Mã': 'Stock_Code',
#         'Tên công ty': 'Company_Name',
#         'Vốn hóa thị trường\nĐơn vị: VND': 'Market_Cap',
#         'STT': 'Index',
#         'SourceFile': 'Source_File'
#     })

#     # 3. CHUYỂN KIỂU DỮ LIỆU
#     # Chuyển đổi kiểu dữ liệu
#     df['Free_Float'] = pd.to_numeric(df['Free_Float'], errors='coerce')
#     df['Luu_Hanh'] = pd.to_numeric(df['Luu_Hanh'], errors='coerce')
#     df['Niem_Yet'] = pd.to_numeric(df['Niem_Yet'], errors='coerce')
#     df['Market_Cap'] = pd.to_numeric(df['Market_Cap'], errors='coerce')
#     df['Date'] = pd.to_datetime(df['Date'], errors='coerce')

#     # Pre-process data
#     df_long = df.copy() 

#     # Convert data types and handle missing values
#     df_long['Free_Float'] = pd.to_numeric(df_long['Free_Float'], errors='coerce').fillna(0)
#     df_long['Luu_Hanh'] = pd.to_numeric(df_long['Luu_Hanh'], errors='coerce').fillna(0)
#     df_long['Niem_Yet'] = pd.to_numeric(df_long['Niem_Yet'], errors='coerce').fillna(0)
#     df_long['Market_Cap'] = pd.to_numeric(df_long['Market_Cap'], errors='coerce').fillna(0)  # Fill missing Market_Cap with 0
#     df_long['Date'] = pd.to_datetime(df_long['Date'], errors='coerce')

#     # Filter invalid data
#     df_long = df_long[df_long['Luu_Hanh'] > 0]
#     df_long = df_long[df_long['Free_Float'] <= df_long['Luu_Hanh']]
#     df_long = df_long[df_long['Free_Float'] >= 0]

#     print("\n=== After Pre-Processing ===")
#     print(df_long.head())
#     print("\n=== Debug Data ===")
#     print("Total Free_Float = 0:", (df_long['Free_Float'] == 0).sum())
#     print("Total Luu_Hanh = 0:", (df_long['Luu_Hanh'] == 0).sum())
#     print("Free_Float sum by Date:", df_long.groupby('Date')['Free_Float'].sum().head())
#     print("Luu_Hanh sum by Date:", df_long.groupby('Date')['Luu_Hanh'].sum().head())

#     # Calculate industry Free Float ratio
#     nghanh_freefloat = df_long.groupby('Date').apply(
#         lambda x: x['Free_Float'].sum() / x['Luu_Hanh'].sum() if x['Luu_Hanh'].sum() != 0 else 0
#     )
#     nghanh_freefloat = pd.DataFrame(nghanh_freefloat, columns=['Tỷ lệ FF']).reset_index()

#     # Print results
#     print(f"\n=== TỶ LỆ FREE FLOAT NGÀNH ({tennganh}) ===")
#     print(nghanh_freefloat.head(10))

#     # Save results
#     output = f'ty_le_freefloat_{tennganh}'
#     nghanh_freefloat.to_excel(f'data\processed\{output}.xlsx', index=False)

#     return nghanh_freefloat  # Return DataFrame for further use if needed

# # Example usage
# if __name__ == "__main__":
#     list_nganh = ['hangtieudung','duocphamyte','taichinh','nganhang','congnghiep','dichvutieudung',
#                   'tienichcd','nguyenvatlieu','vienthong','cntt']
#     result = process_free_float_data('duocphamyte')
#     if result is not None:
#         print("Processing completed. Results saved to files.")

In [ ]:
# Xu ly rieng dau khi

# Step 1: Define file path
daukhi_path = r'data\raw\daukhi\daukhi.xlsx'

# Step 2: Read the Excel file
df_daukhi = pd.read_excel(daukhi_path, header=7)
df_daukhi = pd.DataFrame(df_daukhi)
# Print initial structure
print("=== First 5 rows ===")
print(df_daukhi.head())
print("\n=== Column names ===")
print(df_daukhi.columns.tolist())
# Dynamically find STT and Date columns
columns = df_daukhi.columns.tolist()
stt_col = next((col for col in columns if 'STT' in str(col).upper()), None)
date_col = next((col for col in columns if 'Ngày' in str(col)), None)

if not stt_col or not date_col:
    print(f"Error: Could not find STT or Ngày columns. Available columns: {columns}")
    exit()

print(f"Using STT column: {stt_col}")
print(f"Using Date column: {date_col}")

# Define companies and reshape with positional mapping
companies = ['BSR', 'OIL', 'PEQ', 'PLX', 'POS', 'PTV', 'PVB', 'PVC', 'PVD', 'PVE', 'PVS', 'TOS', 'PETROVN', 'BTPETROCHE']
columns_to_keep = [stt_col, date_col]
melted_dfs = []

for company in companies:
    company_cols = [col for col in columns if company in str(col)]
    if company_cols and len(company_cols) >= 4:  # Ensure at least 4 columns per company
        temp_df = df_daukhi[columns_to_keep + company_cols].copy()
        # Positional renaming (assuming order: Free_Float, Luu_Hanh, Niem_Yet, Market_Cap)
        rename_dict = {
            company_cols[0]: 'Free_Float',  # First column is Free Float
            company_cols[1]: 'Luu_Hanh',    # Second column is Outstanding Shares
            company_cols[2]: 'Niem_Yet',    # Third column is Listed Shares
            company_cols[3]: 'Market_Cap'   # Fourth column is Market Cap
        }
        temp_df = temp_df.rename(columns=rename_dict)
        # Debug: Print renamed columns
        print(f"\nCompany: {company}, Renamed columns: {temp_df.columns.tolist()}")
        temp_df['Stock_Code'] = company
        temp_df = temp_df[[stt_col, date_col, 'Stock_Code', 'Free_Float', 'Luu_Hanh', 'Niem_Yet', 'Market_Cap']]
        melted_dfs.append(temp_df)

# Concatenate
df_long = pd.concat(melted_dfs, ignore_index=True)
df_long = df_long.rename(columns={stt_col: 'Index', date_col: 'Date'})

print("=== Long Format Data ===")
print(df_long.head())

# Drop rows with all critical metadata missing
    initial_rows = len(df)
    df = df.dropna(subset=['STT','Mã', 'Tên công ty', 'Ngày'], how='all')
    print(f"Dropped {initial_rows - len(df)} rows with missing Stock_Code, Company_Name, and Date.")

    # Standardize column names
    df = df.rename(columns={
        'Số CP Free Float\nĐơn vị: Cổ phiếu': 'Free_Float',
        'Số CP lưu hành hiện thời\nĐơn vị: Cổ phiếu': 'Luu_Hanh',
        'Số CP niêm yết\nĐơn vị: Cổ phiếu': 'Niem_Yet',
        'Ngày': 'Date',
        'Mã': 'Stock_Code',
        'Tên công ty': 'Company_Name',
        'Vốn hóa thị trường\nĐơn vị: VND': 'Market_Cap',
        'STT': 'Index',
        'SourceFile': 'Source_File'
    })

    # Convert data types
    numeric_cols = ['STT','Free_Float', 'Luu_Hanh', 'Niem_Yet', 'Market_Cap']
    for col in numeric_cols:
        if col in df.columns:
            df[col] = pd.to_numeric(df[col], errors='coerce')
    df['Date'] = pd.to_datetime(df['Date'], errors='coerce')

    # Merge external Market_Cap if provided
    if external_market_cap_file:
        print(f"Loading external Market_Cap from {external_market_cap_file}")
        external_data = pd.read_csv(external_market_cap_file)
        external_data['Date'] = pd.to_datetime(external_data['Date'], errors='coerce')
        external_data = external_data[['Stock_Code', 'Date', 'Market_Cap']]
        df = df.drop(columns=['Market_Cap'], errors='ignore')
        df = df.merge(external_data, on=['Stock_Code', 'Date'], how='left')
        print("\n=== External Market_Cap Merged ===")
        print(df[['Stock_Code', 'Date', 'Market_Cap']].head())

    # Print initial structure
    print("\n=== First 5 rows ===")
    print(df.head())
    print("\n=== Column names and info ===")
    print(df.info())

    # Log missing data
    print("\n=== Missing Data Summary ===")
    for col in df.columns:
        missing_count = df[col].isna().sum()
        if missing_count > 0:
            print(f"Column {col}: {missing_count} missing values")

    # Investigate missing Free_Float
    if df['Free_Float'].isna().any():
        print("\n=== Stocks with Missing Free_Float ===")
        print(df[df['Free_Float'].isna()]['Stock_Code'].value_counts().head())
        print("\n=== Dates with Missing Free_Float ===")
        print(df[df['Free_Float'].isna()]['Date'].value_counts().sort_index().head())

    # Sort by Stock_Code and Date for consistent filling
    df = df.sort_values(['Stock_Code', 'Date'])

    # Apply ffill within Stock_Code
    for col in ['Free_Float', 'Luu_Hanh', 'Market_Cap']:
        if col in df.columns and df[col].isna().any():
            print(f"Applying ffill to {col} within Stock_Code.")
            df[col] = df.groupby('Stock_Code')[col].ffill()

    # Drop remaining missing values
    missing_free_float = df['Free_Float'].isna().sum()
    missing_market_cap = df['Market_Cap'].isna().sum()
    missing_luu_hanh = df['Luu_Hanh'].isna().sum()
    if missing_free_float > 0 or missing_market_cap > 0 or missing_luu_hanh > 0:
        print(f"Dropping {missing_free_float} rows with missing Free_Float, {missing_market_cap} rows with missing Market_Cap, and {missing_luu_hanh} rows with missing Luu_Hanh.")
        df = df.dropna(subset=['Free_Float', 'Market_Cap', 'Luu_Hanh'])

    # Handle invalid dates
    invalid_dates = df['Date'].isna().sum()
    if invalid_dates > 0:
        print(f"Dropping {invalid_dates} rows with invalid dates.")
        df = df.dropna(subset=['Date'])

    # Filter invalid data
    df_long = df[df['Luu_Hanh'] > 0]
    dropped_luu_hanh = len(df) - len(df_long)
    if dropped_luu_hanh > 0:
        print(f"Dropped {dropped_luu_hanh} rows with Luu_Hanh <= 0.")
    df_long = df_long[df_long['Free_Float'] <= df_long['Luu_Hanh']]
    df_long = df_long[df_long['Free_Float'] >= 0]

    # Log data retention
    print(f"\nRemaining rows after filtering: {len(df_long)} ({len(df_long)/initial_rows*100:.2f}% of original)")
    print(f"Unique stocks: {df_long['Stock_Code'].nunique()}")
    print(f"Unique dates: {df_long['Date'].nunique()}")

    print("\n=== After Pre-Processing ===")
    print(df_long.head())
    print("\n=== Debug Data ===")
    print("Total Free_Float = 0:", (df_long['Free_Float'] == 0).sum())
    print("Total Luu_Hanh = 0:", (df_long['Luu_Hanh'] == 0).sum())
    print("Free_Float sum by Date:", df_long.groupby('Date')['Free_Float'].sum().head())
    print("Luu_Hanh sum by Date:", df_long.groupby('Date')['Luu_Hanh'].sum().head())
    print("\n=== Missing Data After Preprocessing ===")
    print(df_long.isna().sum())

    # Calculate industry Free Float ratio
    nghanh_freefloat = df_long.groupby('Date').apply(
        lambda x: x['Free_Float'].sum() / x['Luu_Hanh'].sum() if x['Luu_Hanh'].sum() != 0 else 0
    )
    zero_luu_hanh_dates = df_long.groupby('Date').filter(lambda x: x['Luu_Hanh'].sum() == 0)['Date'].unique()
    for date in zero_luu_hanh_dates:
        print(f"Warning: Zero Luu_Hanh sum for date {date}")
    nghanh_freefloat = pd.DataFrame(nghanh_freefloat, columns=['Tỷ lệ FF']).reset_index()

# Convert data types
df_long['Free_Float'] = pd.to_numeric(df_long['Free_Float'], errors='coerce').fillna(0)
df_long['Luu_Hanh'] = pd.to_numeric(df_long['Luu_Hanh'], errors='coerce').fillna(0)
df_long['Niem_Yet'] = pd.to_numeric(df_long['Niem_Yet'], errors='coerce').fillna(0)
df_long['Market_Cap'] = pd.to_numeric(df_long['Market_Cap'], errors='coerce').fillna(0)
df_long['Date'] = pd.to_datetime(df_long['Date'], errors='coerce')

# Filter invalid data
df_long = df_long[df_long['Luu_Hanh'] > 0]
df_long = df_long[df_long['Free_Float'] <= df_long['Luu_Hanh']]
df_long = df_long[df_long['Free_Float'] >= 0]

print("\n=== After Pre-Processing ===")
print(df_long.head())
print("\n=== Debug Data ===")
print("Total Free_Float = 0:", (df_long['Free_Float'] == 0).sum())
print("Total Luu_Hanh = 0:", (df_long['Luu_Hanh'] == 0).sum())
print("Free_Float sum by Date:", df_long.groupby('Date')['Free_Float'].sum().head())
print("Luu_Hanh sum by Date:", df_long.groupby('Date')['Luu_Hanh'].sum().head())

# Calculate industry Free Float ratio
nganh_freefloat = df_long.groupby('Date').apply(
    lambda x: x['Free_Float'].sum() / x['Luu_Hanh'].sum() if x['Luu_Hanh'].sum() != 0 else 0
)
nganh_freefloat = pd.DataFrame(nganh_freefloat, columns=['Tỷ lệ FF']).reset_index()

# Print results
print("\n=== TỶ LỆ FREE FLOAT NGÀNH ===")
print(nganh_freefloat.head(10))
# Step 8: Save results
nganh_freefloat.to_excel('data\processed\ty_le_freefloat_daukhi.xlsx', index=False)


=== First 5 rows ===
   STT       Ngày                                 BSR  \
0  NaN        NaT  Số CP Free Float\nĐơn vị: Cổ phiếu   
1    1 2025-10-01                           310049961   
2    2 2025-09-30                           310049961   
3    3 2025-09-29                           310049961   
4    4 2025-09-26                           310049961   

                                        BSR.1  \
0  Số CP lưu hành hiện thời\nĐơn vị: Cổ phiếu   
1                                  3100499616   
2                                  3100499616   
3                                  3100499616   
4                                  3100499616   

                             BSR.2                             BSR.3  \
0  Vốn hóa thị trường\nĐơn vị: VND  Số CP niêm yết\nĐơn vị: Cổ phiếu   
1                   82318264804800                        3100499616   
2                   81233089939200                        3100499616   
3                   83403439670400                   

C:\Users\caida\AppData\Local\Temp\ipykernel_7376\1475278332.py:77: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  nganh_freefloat = df_long.groupby('Date').apply(



=== TỶ LỆ FREE FLOAT NGÀNH ===
        Date  Tỷ lệ FF
0 2008-12-31       0.0
1 2009-12-31       0.0
2 2010-12-21       0.0
3 2010-12-22       0.0
4 2010-12-23       0.0
5 2010-12-24       0.0
6 2010-12-27       0.0
7 2010-12-28       0.0
8 2010-12-29       0.0
9 2010-12-30       0.0
